In [1]:
import pandas as pd
import numpy as np
import timeit
import test_performance_helper as tph
import sklearn.metrics
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
gene_of_interest = 'AR'

In [3]:
X = pd.read_csv('../tmp_data/feature_matrix.txt', delimiter = '\t', header=0, index_col=0)
y = pd.read_csv('../tmp_data/label_vector.txt', delimiter = '\t', header=0, index_col=0)

In [4]:
X_cnv = pd.read_csv('../tmp_data/feature_matrix_cnv.txt', delimiter = '\t', header=0, index_col=0)
y_cnv = pd.read_csv('../tmp_data/label_vector_cnv.txt', delimiter = '\t', header=0, index_col=0)

# read the file with best hyperparameters
best_hp = pd.read_csv('../results/'+str(gene_of_interest)+'/best_hyper_param.txt', delimiter = '\t', header=0)

# read the file containing the best setting
best_setting = pd.read_csv('/projects/fkeshavarz_prj/fkeshavarz_scratch/data/best_classification_setting.tsv', delimiter = '\t', header=0)

In [ ]:
tcga_t_type = pd.read_csv('/projects/fkeshavarz_prj/fkeshavarz_scratch/data/tcga/tcga_t_type.tsv', delimiter = '\t', header=0)
pog_t_type = pd.read_csv('/projects/fkeshavarz_prj/fkeshavarz_scratch/data/pog/hg38/pog_t_type.tsv', delimiter = '\t', header=0)

# read expression data of samples with non-impactful mutations
tcga_tpm_not_impactful_mut = pd.read_csv('../tmp_data/tcga_tpm_not_impactful_mut.txt', delimiter = '\t', header=0, index_col=0)
pog_tpm_not_impactful_mut = pd.read_csv('../tmp_data/pog_tpm_not_impactful_mut.txt', delimiter = '\t', header=0, index_col=0)

# read processed mutation data
tcga_all_mut = pd.read_csv('../tmp_data/tcga_mut_prcssd.tsv', delimiter = '\t', header=0)
pog_all_mut = pd.read_csv('../tmp_data/pog_mut_prcssd.tsv', delimiter = '\t', header=0)

In [ ]:
tumour_type_dict = {'APC':['COADREAD'],
                    'AR':['KIRP'],
                    'ARID1A':['BRCA', 'LGG', 'LIHC', 'PCPG', 'UVM'],
                    'ASXL1':['BRCA','CESC'],
                    'ATM':['BRCA', 'CESC', 'PCPG'],
                    'ATR':['HNSC', 'KIRP', 'PCPG'],
                    'ATRX':['LGG'],
                    'BRAF':['COADREAD', 'SKCM', 'THCA'],
                    'BRCA1':['BRCA', 'KICH', 'KIRP', 'UCEC'],
                    'BRCA2':['BRCA'],
                    'CDH1':['KIRP', 'LIHC', 'PRAD', 'SARC', 'THYM', 'UCEC', 'UVM'],
                    'CDK12':['BRCA', 'KICH', 'KIRP', 'SARC', 'UCEC'],
                    'CDKN2A':['HNSC', 'KIRC', 'LGG', 'MESO', 'PAAD', 'UCEC'],
                    'CTCF':['KIRP', 'LIHC', 'PRAD', 'SARC', 'THYM', 'UVM'],
                    'CTNNB1':['BRCA', 'CESC', 'HNSC', 'KIRC', 'LIHC', 'PCPG', 'UVM'],
                    'EGFR':['COADREAD', 'HNSC', 'KIRP', 'LGG', 'STAD', 'THCA'],
                    'EP300':['BRCA', 'PCPG', 'THCA', 'UCEC'],
                    'ERBB4':['CESC', 'PAAD'],
                    'EZH2':['COADREAD', 'KIRP', 'LGG', 'THCA', 'THYM'],
                    'FBXW7':['LIHC', 'MESO', 'UCEC'],
                    'FLT3':['UCEC'],
                    'GATA3':['LGG', 'SARC'],
                    'KDM6A':['KIRP', 'UCEC'],
                    'KEAP1':['GBM', 'LUAD', 'STAD', 'UCEC'],
                    'KIT':['ACC', 'KICH'],
                    'KRAS':['COADREAD', 'PAAD'],
                    'MAP3K1':['BLCA', 'HNSC', 'PRAD', 'STAD'],
                    'MECOM':['UCEC', 'UVM'],
                    'MTOR':['BRCA', 'LGG', 'PCPG'],
                    'NCOR1':['BRCA', 'COADREAD', 'KICH', 'KIRP', 'LIHC', 'PAAD', 'PCPG', 'UCEC'],
                    'NF1':['BRCA', 'KICH', 'KIRP', 'PCPG', 'SARC', 'UCEC'],
                    'NFE2L2':['KICH', 'THCA'],
                    'NOTCH1':['KIRC'],
                    'NRAS':['LGG', 'PCPG'],
                    'NSD1':['BLCA', 'HNSC', 'KIRC'],
                    'PBRM1':['BRCA', 'CESC', 'HNSC', 'KIRC', 'KIRP', 'PCPG', 'UVM'],
                    'PDGFRA':['KICH'],
                    'PIK3CA':['KIRP', 'PCPG', 'UVM'],
                    'PIK3R1':['BLCA', 'HNSC', 'STAD'],
                    'POLQ':['BRCA'],
                    'PTEN':['LGG', 'PRAD', 'SARC', 'SKCM'],
                    'RB1':['BRCA', 'CESC', 'COADREAD', 'GBM', 'LGG', 'LIHC', 'PRAD', 'SARC'],
                    'SETBP1':['HNSC', 'PRAD'],
                    'SETD2':['BRCA', 'CESC', 'HNSC', 'KIRC', 'KIRP', 'PCPG', 'UVM'],
                    'SF3B1':['UVM'],
                    'SMAD4':['STAD'],
                    'SPOP':['BRCA', 'KICH', 'KIRP'],
                    'STAG2':['KIRP'],
                    'TET2':['LIHC', 'MESO'],
                    'TP53':['BLCA', 'BRCA', 'COADREAD', 'HNSC', 'LGG', 'LIHC', 'LUAD', 'STAD', 'UCEC']}

In [ ]:
all_t_type = pd.concat([tcga_t_type, pog_t_type], axis=0)
spcfc_t_types = all_t_type[all_t_type.tumour_type_abbv.isin(tumour_type_dict[gene_of_interest])]
spcfc_p_ids = spcfc_t_types[spcfc_t_types.tumour_type_abbv.isin(tumour_type_dict[gene_of_interest])].p_id

# filter X and y based on above ids
if best_setting[best_setting.gene==gene_of_interest].best_setting.iloc[0] == 'SNV_only':
    X_new = X.loc[(X.index.isin(spcfc_p_ids)==True),]
    y_new = y.loc[(y.index.isin(spcfc_p_ids)==True),]
elif best_setting[best_setting.gene==gene_of_interest].best_setting.iloc[0] == 'SNV_CNV':
    X_new = X_cnv.loc[(X_cnv.index.isin(spcfc_p_ids)==True),]
    y_new = y_cnv.loc[(y_cnv.index.isin(spcfc_p_ids)==True),]

In [ ]:
hps = best_hp.iloc[2,][0].split(',')

for i in range(len(hps)):
    if 'max_depth' in hps[i]:
        best_max_depth = int(hps[i].split(':')[1])
    elif 'max_features' in hps[i]:
        if 'sqrt' in hps[i].split(':')[1]:
            best_max_features = 'sqrt'
        else:
            best_max_features = float(hps[i].split(':')[1])
    elif 'max_samples' in hps[i]:
        best_max_samples = float(hps[i].split(':')[1])
    elif 'min_samples_leaf' in hps[i]:
        best_min_samples_leaf = int(hps[i].split(':')[1])
    elif 'min_samples_split' in hps[i]:
        best_min_samples_split = int(hps[i].split(':')[1])

In [ ]:
clf = RandomForestClassifier(n_estimators=3000, max_depth=best_max_depth, max_features=best_max_features, 
                             max_samples=best_max_samples, min_samples_split=best_min_samples_split, 
                             min_samples_leaf=best_min_samples_leaf, n_jobs=40)

skf = StratifiedKFold(n_splits=5, shuffle=True)

# random forest performance on tcga and pog
y_new = y_new.y
all_pred_df, all_prob, true_label_prob = tph.test_performance_5_fold_CV(clf, skf, X_new, y_new)

In [ ]:
print(confusion_matrix(all_pred_df.status, all_pred_df.predict, labels=['mut','wt']))
print(classification_report(all_pred_df.status, all_pred_df.predict))